In [41]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import csv
import numpy as np
from bs4 import BeautifulSoup
import schedule
import time
from datetime import datetime
#Thư viện để dừng schedule
import sys
import os
import signal
#thu vien thread chay da luong
import threading
#cai dat va chay driver chrome
url = "https://iboard.ssi.com.vn/bang-gia/hose"
driver = webdriver.Chrome("C:\chromedriver.exe")
driver.get(url)
#Đọc các mã chứng khoán trong file Ma_chung_khoan.txt
Ma_chung_khoan=open("Ma_chung_khoan.txt","r")
Ma_CK=Ma_chung_khoan.readlines()
Ma_chung_khoan.close() 
#loại bỏ dấu  ","
Ma_CK=Ma_CK[0].split(',')
for mck in Ma_CK:
    #Đường link file cào, sys.path[0] là đường dẫn thư mục của code         
    save_path = sys.path[0].replace('\\','/')+'/DATA_HOSE/'+mck
    #Kiểm tra xem đã có folder đó chưa, nếu chưa thì tạo mới
    if not os.path.exists(save_path):
        os.makedirs(save_path)



In [43]:
#Xử lí lưu file
def xu_ly_file(a,mck):
        #Đường link file cào, sys.path[0] là đường dẫn thư mục của code 
        save_path = sys.path[0].replace('\\','/')+'/DATA_HOSE/'+mck
        #Tên file.csv: Mã chứng khoán + ngày tháng năm
        completeName = os.path.join(save_path,mck+"_"+datetime.today().strftime('%d_%m_%Y')+".csv")
        #sửa mode w+ thành mode a để viết được vào cuối file
        file = open(completeName,"a",)
        with file:
            write = csv.writer(file)
            write.writerows(np.array(a).reshape(-1,27))
#Crawl Mã
def job():
    start_time = time.time()
    #download sourse html page
    page_source = BeautifulSoup(driver.page_source,"xml")
    p=str(datetime.now())   
    for mck in Ma_CK:
        a=[]
        info_tr=page_source.find('tr', id=mck)
        info_td=info_tr.findAll('td')
        a.extend([p])
        for i in range(26):
            if info_td[i].get_text()=='':
                a.extend("N")
            else:
                a.append(info_td[i].get_text())
        xu_ly_file(a,mck)
        print("--- %s seconds ---" % (time.time() - start_time)) 

In [44]:
#lớp cào chia luồng
class crawler(threading.Thread):
    
    def __init__(self, list_lock):
        threading.Thread.__init__(self)
        self._list_lock = list_lock	# Lock for list_num

    def run(self):
        while True:
        	# request to access shared resource
            # if there are many threads acquiring Lock, only one thread get the Lock
            # and other threads will get blocked
            #lấy khóa
            self._list_lock.acquire()
            job()
            #trả khóa
            self._list_lock.release()
    
    
def parallel_primes(num_threads=None):
    """Parallel count number of prime from 2 to n

    Count prime numbers from 2 to n using num_threads threads
    If num_threads is None, using os.cpu_count()

    """
    list_lock = threading.Lock()
    threads = []

    if num_threads is None:
        try:
            num_threads = 2
        except AttributeError:
            num_threads = 4

    elif num_threads < 1:
        raise ValueError('num_threads must be > 0')

    for i in range(num_threads):
        thread = crawler(list_lock)
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()


schedule.every().day.at("16:58:01").do(parallel_primes)
schedule.every().day.at("11:44").do(exit)
#vòng while này kiểm tra mỗi giây coi đúng với tgian mình muốn để thực hiện hay chưa
while True:
    schedule.run_pending()
    time.sleep(1)

--- 4.698205232620239 seconds ---
--- 4.707198858261108 seconds ---
--- 4.7131969928741455 seconds ---
--- 4.719191551208496 seconds ---
--- 4.721190452575684 seconds ---
--- 1.100964069366455 seconds ---
--- 1.1029620170593262 seconds ---
--- 1.1049611568450928 seconds ---
--- 1.1099586486816406 seconds ---
--- 1.1139564514160156 seconds ---
--- 1.107950210571289 seconds ---
--- 1.1119494438171387 seconds ---
--- 1.1179468631744385 seconds ---
--- 1.1219449043273926 seconds ---
--- 1.1249427795410156 seconds ---
--- 1.3242368698120117 seconds ---
--- 1.327235460281372 seconds ---
--- 1.3312337398529053 seconds ---
--- 1.3372294902801514 seconds ---
--- 1.3442246913909912 seconds ---
--- 1.9628708362579346 seconds ---
--- 1.9688680171966553 seconds ---
--- 1.973865032196045 seconds ---
--- 1.9798626899719238 seconds ---
--- 1.985858678817749 seconds ---
--- 1.4889740943908691 seconds ---
--- 1.495967149734497 seconds ---
--- 1.5019636154174805 seconds ---
--- 1.508960247039795 seconds 